# A Guide to Streamlining CFD Simulations and Reporting with Ansys Fluent, Dynamic Reporting, and PyAnsys.

## Prerequisite

- [Ansys Fluent](https://www.ansys.com/products/fluids/ansys-fluent)
- [Ansys Dynamic Reporting](https://nexusdemo.ensight.com/docs/en/html/Nexus.html)
- [Pyfluent](https://pypi.org/project/ansys-fluent-core/) 
- [PyDynamicReporting](https://pypi.org/project/ansys-dynamicreporting-core/)

1. **Install Python (if not already installed):**
   
   Ensure you have Python installed on your system. You can download Python from the [official website](https://www.python.org/downloads/).  The recommended version for the setup is [Python 3.10](https://www.python.org/downloads/) 

2. **Create a virtual environment (if not already installed):**
   
   ```
      python -m venv .venv      
   ```

3. **Activate the virtual environment:**

  * For Windows CMD:

    ```bash

      .venv\Scripts\activate.bat
    ```

  * For Windows Powershell:

    ```bash

      .venv\Scripts\Activate.ps1
    ```

From now on, all the commands must be executed within the virtual environment.

## Fluent Simulation Workflow for Example Case

### Reading Ansys Fluent Simulation data using PyFluent.

**Install PyFluent & PyADR modules**

Before that upgrade pip

In [ ]:
!pip install --upgrade pip 

In [ ]:
!pip install ansys-fluent-core
!pip install ansys-dynamicreporting-core

**Import required libraries/modules**

In [ ]:
from pathlib import Path
import ansys.fluent.core as pyfluent
from ansys.fluent.core import examples

**Downloading cas/dat file**

In [ ]:
save_path = Path(pyfluent.EXAMPLES_PATH)

import_filename = examples.download_file(
    "mixing_elbow.cas.h5",
    "pyfluent/mixing_elbow",
)  

examples.download_file(
    "mixing_elbow.dat.h5",
    "pyfluent/mixing_elbow",
    save_path=save_path,
)

**Launch Fluent session**

In [ ]:
session = pyfluent.launch_fluent(
    show_gui=True, processor_count=4, product_version="23.1.0"
)

**Reading case and data file**

In [ ]:
session.file.read_case_data(file_type='case-data', file_name=import_filename)

**Create a velocity vectors**

In [ ]:
session.results.graphics.vector["velocity_vector_symmetry"] = {}
session.results.graphics.vector["velocity_vector_symmetry"].print_state()
session.results.graphics.vector["velocity_vector_symmetry"].field = "temperature"
session.results.graphics.vector["velocity_vector_symmetry"].surfaces_list = [
    "symmetry-xyplane",
]
session.results.graphics.vector["velocity_vector_symmetry"].scale.scale_f = 4
session.results.graphics.vector["velocity_vector_symmetry"].style = "arrow"

**Generate the Report using the PyFluent API**

In [ ]:
session.report.simulation_reports.generate_simulation_report(
    report_name="Mixing elbow simulation"
)
session.report.simulation_reports.export_simulation_report_as_pdf(
    report_name="Mixing elbow simulation", file_name_path=save_path
)

**Close the Ansys Fluent Session**

In [ ]:
session.exit()

With the above steps, the process of accessing simulation data and initiating an Ansys Fluent session using PyFluent is discussed. This includes creating a velocity contour at the symmetry plane and generating a comprehensive simulation report using Ansys Dynamic Reporting. Furthermore, exporting this report as a PDF is demonstrated. The subsequent section guides users through utilizing PyDynamic Reporting to connect with FluentReportServer and obtain the report URL, enabling its integration with any framework using an iframe source.

>**Note:** The FluentReportServer directory is created by Ansys Fluent at current working directory. It contains the Ansys Dynamic Reporting database and relevant database files.

## Ansys PyDynamicReporting Workflow

### Connecting with FluentReportServer (ADR Service) using PyDynamicReporting library

**Import required libraries/modules**

In [ ]:
from pathlib import Path
import ansys.dynamicreporting.core as adr

**Connect to Fluent Report Server (ADR Service)**

In [ ]:
data_base_ref = r'FluentReportServer'
ansys_installation = r"C:\Program Files\ANSYS Inc\v231"

adr_service = adr.Service(
    ansys_installation=ansys_installation,
    db_directory=data_base_ref,
)
session_guid = adr_service.start(create_db=False)

**Get the port of ADR service and server object for Report Updates**

In [ ]:
adr_service._port
server = adr_service.serverobj

**Updating the existing report template**

Here, updating the Organizations branding logo and creating data item for it, adding title, Table of Contents. 

In [ ]:
## Place here the name of the top-level report
myr = adr_service.get_report("Mixing elbow simulation")
root_template = myr.report
report_tags = root_template.get_tags()

template_0=server.create_template(name="Top", parent=root_template, report_type="Layout:panel")
template_0.params='{"properties": {"TOCItem": "0"}}'
template_0.set_tags(report_tags)
server.put_objects(template_0)
server.put_objects(root_template)

template_1=server.create_template(name="Logo", parent=template_0, report_type="Layout:basic")
template_1.params='{"properties": {"TOCItem": "0"}}'
template_1.set_filter("A|i_type|eq|image;A|i_name|eq|UserLogo;")
template_1.set_tags(report_tags)
server.put_objects(template_1)
server.put_objects(template_0)
server.put_objects(root_template)

template_2=server.create_template(name="Title", parent=template_0, report_type="Layout:basic")
template_2.params='{"properties": {"TOCItem": "0"}}'
template_2.set_filter("A|i_type|eq|html;A|i_name|eq|ReportTitle;")
template_2.set_tags(report_tags)
server.put_objects(template_2)
server.put_objects(template_0)
server.put_objects(root_template)

template_3=server.create_template(name="Header", parent=template_0, report_type="Layout:basic")
template_3.params='{"properties": {"TOCItem": "0"}, "HTML": "", "skip_empty": 1}'
template_3.set_filter("A|i_type|eq|table;A|i_name|eq|HeaderTableItem;")
template_3.set_tags(report_tags)
server.put_objects(template_3)
server.put_objects(template_0)
server.put_objects(root_template)

template_4=server.create_template(name="Table of Contents", parent=template_0, report_type="Layout:toc")
template_4.params='{"HTML": "<h3>Table of Contents</h3>", "TOCitems": 1, "TOCtables": 0, "TOCfigures": 0}'
template_4.set_filter("A|i_name|cont|_nothing_selected_;")
template_4.set_tags(report_tags)
server.put_objects(template_4)
server.put_objects(template_0)
server.put_objects(root_template)

# Re-order the template children so the new one goes on top
root_template.children_order = template_0.guid+',' + root_template.children_order.replace(template_0.guid+',', '')
root_template.reorder_children()
server.put_objects(root_template)


## Now add the Logo the customer wants to use
logo_img = adr_service.create_item(obj_name='UserLogo')
logo_img.item.dataset = report_tags.split('=')[-1]
logo_img.item_image = r'logo.png'

>**Note:** Use any image file to upload as logo to create item_image, recommended to use PNG, JPEG format.

**Get the URL of the report**

In [ ]:
report_url = adr_service.get_report("Mixing elbow simulation")
report_url.get_url()

The URL returned by the ADR service can be easily embedded with any frontend technology.

e.g., with Plotly Dash

To create a simple Dash web application that renders a URL with an iframe using Dash Bootstrap Components (dbc), you'll need to have Dash and dbc installed. You can install them using pip if you haven't already:

**Install Plotly Dash**

In [ ]:
!pip install dash 
!pip install dash-bootstrap-components

**Ploty Dash Example to show case embedded ADR report using URL obtained from ADR Server**

In [ ]:
import dash
import dash_bootstrap_components as dbc
from dash import html, dcc

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    dbc.Card(
        [
            html.H1("Report Viewer", className="display-3"),
            html.P(
                "Enter a URL to render it in an iframe below:",
                className="lead",
            ),
            dcc.Input(id="url-input", type="text", placeholder="Enter URL"),
            html.Div(id="iframe-container"),
        ],
    )
])

@app.callback(
    dash.dependencies.Output("iframe-container", "children"),
    [dash.dependencies.Input("url-input", "value")]
)
def update_iframe(url):
    if url:
        return html.Iframe(src=url, width="100%", height="500")
    return html.Div()

if __name__ == '__main__':
    app.run_server(debug=True)

Save this code in a Python file (e.g., app.py) and run it. You can access the web application in your browser at http://localhost:8050, and it will allow you to enter a URL to render in an iframe.

**Stop ADR Service**

In [ ]:
adr_service.stop()

## Conclusion

The integration of Ansys Fluent, Ansys Dynamic Reporting, and their respective client libraries, PyFluent and PyDynamicReporting, not only simplifies simulation and reporting processes but also offers numerous application and integration opportunities. By exporting reports in different formats, users can tailor the generated insights to suit their particular requirements, such as interactive web applications, widespread insight sharing, SaaS applications, or sophisticated data analytics. These tools enable engineers and researchers to fully utilize their simulation data in today's technology-driven world.

## Additional Resources
[Ansys Fluent](https://www.ansys.com/products/fluids/ansys-fluent)

[Ansys PyFluent Documentation](https://fluent.docs.pyansys.com/version/stable)

[Ansys PyDynamic Reporting Documentation](https://dynamicreporting.docs.pyansys.com/version/stable/)

[Plotly Dash](https://dash.plotly.com/)